In [84]:

from ta.trend import ADXIndicator
from sklearn.linear_model import LogisticRegression
import pandas as pd
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
import pandas_ta as ta
from ta.trend import MACD

c:\Users\vamsi\OneDrive\Desktop\Documents\studies\complete stock market predictor\venv\Lib\site-packages\pandas_ta\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [156]:
x=pd.read_csv("C:/Users/vamsi/OneDrive/Desktop/Documents/studies/complete stock market predictor/Stock-Market-Predictor/project_root/data/Feature_Engineered_data.csv")
y=pd.read_csv("C:/Users/vamsi/OneDrive/Desktop/Documents/studies/complete stock market predictor/Stock-Market-Predictor/project_root/data/Feature_Engineered_result.csv")

In [157]:
def calculate_SuperTrend(df, period=14, multiplier=3):
    print("HELLO")
    st = ta.supertrend(df['high'], df['low'], df['close'], length=period, multiplier=multiplier)
    st.columns = ['Supertrend', 'Supertrend_Direction', 'Supertrend_Trend', 'Supertrend_Source']
    
    df['Supertrend_Direction'] = st['Supertrend_Direction']
    return df

In [158]:
from sklearn.preprocessing import LabelEncoder
# Label encoding
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

c:\Users\vamsi\OneDrive\Desktop\Documents\studies\complete stock market predictor\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [193]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train, y_test=train_test_split(x,y_encoded,test_size=0.2)

In [194]:
x_train.shape

(8635, 8)

In [188]:
x_test.shape

(2159, 8)

In [196]:
model1=LogisticRegression()
model1.fit(x_train,y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [195]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [197]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(x_train, y_train)

c:\Users\vamsi\OneDrive\Desktop\Documents\studies\complete stock market predictor\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:49:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'mlogloss'


In [198]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

def evaluate(model, name):
    preds = model.predict(x_test)
    print(f"\n{name} Evaluation")
    print("Accuracy:", accuracy_score(y_test, preds))
    print("F1 Score:", f1_score(y_test, preds, average='weighted'))
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
    print("Classification Report:\n", classification_report(y_test, preds))

evaluate(model1, "Logistic Regression")
evaluate(rf_model, "Random Forest")
evaluate(xgb_model, "XGBoost")



Logistic Regression Evaluation
Accuracy: 0.4858730893932376
F1 Score: 0.48524751227137697
Confusion Matrix:
 [[420 140 232]
 [183 269 150]
 [264 141 360]]
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.53      0.51       792
           1       0.49      0.45      0.47       602
           2       0.49      0.47      0.48       765

    accuracy                           0.49      2159
   macro avg       0.49      0.48      0.48      2159
weighted avg       0.49      0.49      0.49      2159


Random Forest Evaluation
Accuracy: 0.6919870310328856
F1 Score: 0.6920162175757117
Confusion Matrix:
 [[546  99 147]
 [101 418  83]
 [135 100 530]]
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.69      0.69       792
           1       0.68      0.69      0.69       602
           2       0.70      0.69      0.70       765

    accuracy                           0.69     

## Majority Voting Ensemble

In [166]:
pred_lr = model1.predict(x_test)
pred_rf = rf_model.predict(x_test)
pred_xgb = xgb_model.predict(x_test)

In [167]:
import numpy as np
from scipy.stats import mode

def majority_vote_ensemble(*predictions):
    """
    Accepts multiple model predictions and returns the majority-voted prediction per row.
    """
    preds = np.array(predictions)
    majority_preds, _ = mode(preds, axis=0, keepdims=False)
    return majority_preds


In [168]:
ensemble_preds = majority_vote_ensemble( pred_rf, pred_xgb)


In [169]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

print("Ensemble Evaluation")
print("Accuracy:", accuracy_score(y_test, ensemble_preds))
print("F1 Score:", f1_score(y_test, ensemble_preds, average='weighted'))
print("Confusion Matrix:\n", confusion_matrix(y_test, ensemble_preds))
print("Classification Report:\n", classification_report(y_test, ensemble_preds))


Ensemble Evaluation
Accuracy: 0.6711440481704493
F1 Score: 0.6702389750191793
Confusion Matrix:
 [[595  90  92]
 [170 393  69]
 [190  99 461]]
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.77      0.69       777
           1       0.68      0.62      0.65       632
           2       0.74      0.61      0.67       750

    accuracy                           0.67      2159
   macro avg       0.68      0.67      0.67      2159
weighted avg       0.68      0.67      0.67      2159



## Fine Tuning Using Optuna for xgboost

In [170]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np


def objective(trial):
    param = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': len(np.unique(y)),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': 42
    }

    model = XGBClassifier(**param, use_label_encoder=False)
    model.fit(x_train, y_train)
    preds = model.predict(x_test)

    return f1_score(y_test, preds, average='macro')


In [171]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can increase trials to 100+


[I 2025-08-21 15:42:23,730] A new study created in memory with name: no-name-449d7abd-d700-4c4f-b759-3cc5ee8981b5
c:\Users\vamsi\OneDrive\Desktop\Documents\studies\complete stock market predictor\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:42:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-08-21 15:42:24,153] Trial 0 finished with value: 0.5114994148928291 and parameters: {'learning_rate': 0.18758720838385604, 'max_depth': 3, 'n_estimators': 586, 'min_child_weight': 9, 'gamma': 3.610542152084153, 'subsample': 0.8376864967361437, 'colsample_bytree': 0.616721054153597, 'reg_alpha': 0.6650727953636482, 'reg_lambda': 0.16898451792008262}. Best is trial 0 with value: 0.5114994148928291.
c:\Users\vamsi\OneDrive\Desktop\Documents\studies\complete stock market predictor\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:42:24] WARNI

In [172]:
print("Best Trial:")
print(study.best_trial)
print("Best Params:")
print(study.best_params)

# Train final model with best params
best_params = study.best_params
best_params.update({
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'num_class': len(np.unique(y)),
    'random_state': 42
})

final_xgb = XGBClassifier(**best_params, use_label_encoder=False)
final_xgb.fit(x_train, y_train)


Best Trial:
FrozenTrial(number=43, state=1, values=[0.7026233186129476], datetime_start=datetime.datetime(2025, 8, 21, 15, 43, 32, 773881), datetime_complete=datetime.datetime(2025, 8, 21, 15, 43, 37, 264296), params={'learning_rate': 0.08027819193577101, 'max_depth': 10, 'n_estimators': 755, 'min_child_weight': 1, 'gamma': 0.008340223266922232, 'subsample': 0.6127918163456542, 'colsample_bytree': 0.8298642873399527, 'reg_alpha': 0.19288641477956842, 'reg_lambda': 0.4817976343873426}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample

c:\Users\vamsi\OneDrive\Desktop\Documents\studies\complete stock market predictor\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:44:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8298642873399527
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'mlogloss'


In [173]:
preds = final_xgb.predict(x_test)

In [174]:
accuracy_score(y_test, preds)

0.7040296433534043

In [60]:
from sklearn.ensemble import VotingClassifier

## Majority Voting Ensemble

In [175]:
ensemble_preds = majority_vote_ensemble( pred_rf, preds,pred_lr)

In [176]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

print("Ensemble Evaluation")
print("Accuracy:", accuracy_score(y_test, ensemble_preds))
print("F1 Score:", f1_score(y_test, ensemble_preds, average='weighted'))
print("Confusion Matrix:\n", confusion_matrix(y_test, ensemble_preds))
print("Classification Report:\n", classification_report(y_test, ensemble_preds))

Ensemble Evaluation
Accuracy: 0.6989346919870311
F1 Score: 0.698667156957811
Confusion Matrix:
 [[553  95 129]
 [136 411  85]
 [126  79 545]]
Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.71      0.69       777
           1       0.70      0.65      0.68       632
           2       0.72      0.73      0.72       750

    accuracy                           0.70      2159
   macro avg       0.70      0.70      0.70      2159
weighted avg       0.70      0.70      0.70      2159



## Voting Ensemble

In [177]:
voting_model = VotingClassifier(
    estimators=[("rf", rf_model), ("xgb", final_xgb)],
    voting="soft",
    weights=[0.45,0.55]   # RF more weight
)

voting_model.fit(x_train, y_train)


c:\Users\vamsi\OneDrive\Desktop\Documents\studies\complete stock market predictor\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:45:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,estimators,"[('rf', ...), ('xgb', ...)]"
,voting,'soft'
,weights,"[0.45, 0.55]"
,n_jobs,None
,flatten_transform,True
,verbose,False
,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1


In [178]:
y_pred=voting_model.predict(x_test)
print("Weighted Voting Accuracy:", accuracy_score(y_test, y_pred))

Weighted Voting Accuracy: 0.7068087077350625


## Stacking Ensemble

In [203]:
# 1. Train base models
rf_preds = rf_model.predict_proba(x_train)
xgb_preds = final_xgb.predict_proba(x_train)

# 2. Concatenate predictions
stacked_input = np.hstack((rf_preds, xgb_preds))  # shape: (n_samples, n_classes*2)

# 3. Train meta model
meta_model = xgb.XGBClassifier()
meta_model.fit(stacked_input, y_train)

# For test
rf_test = rf_model.predict_proba(x_test)
xgb_test = xgb_model.predict_proba(x_test)
stacked_test_input = np.hstack((rf_test, xgb_test))

# Final predictions
ensemble_preds = meta_model.predict(stacked_test_input)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

print("Ensemble Evaluation")
print("Accuracy:", accuracy_score(y_test, ensemble_preds))

Ensemble Evaluation
Accuracy: 0.728281611857342


## Artificial Neural Networks

In [181]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

from sklearn.metrics import classification_report

X_tensor = torch.tensor(x_train.values, dtype=torch.float32)   # x_train is a DataFrame
y_tensor = torch.tensor(y_train, dtype=torch.long)             # y_train is already NumPy


# Create Dataset and DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Define the ANN model
class ANNModel(nn.Module):
    def __init__(self, input_dim):
        super(ANNModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 3),  # 3 classes
        )

    def forward(self, x):
        return self.network(x)

# Initialize
model = ANNModel(input_dim=x_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.006)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

model.eval()
with torch.no_grad():
    X_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
    outputs = model(X_test_tensor)
    predictions = torch.argmax(outputs, dim=1)
    print(classification_report(y_test, predictions))


Epoch 10, Loss: 239.2687
Epoch 20, Loss: 232.9342
Epoch 30, Loss: 227.5411
Epoch 40, Loss: 225.9349
Epoch 50, Loss: 222.9189
Epoch 60, Loss: 221.2022
Epoch 70, Loss: 217.1076
Epoch 80, Loss: 216.7233
Epoch 90, Loss: 217.0525
Epoch 100, Loss: 215.5949
              precision    recall  f1-score   support

           0       0.56      0.48      0.52       777
           1       0.54      0.64      0.59       632
           2       0.58      0.57      0.58       750

    accuracy                           0.56      2159
   macro avg       0.56      0.57      0.56      2159
weighted avg       0.56      0.56      0.56      2159



In [182]:
print(accuracy_score(y_pred=predictions,y_true=y_test))

0.5599814729041223


## Using Traditional Approach

In [92]:
import numpy as np

def apply_strategy_override(df):
    """
    Applies rule-based logic over model predictions and returns a NumPy array of final strategy predictions.

    Parameters:
    - df: DataFrame containing RSI, ADX, Supertrend_Direction
    - model_preds: Numpy array or Series of model predictions like ['Buy', 'Sell', 'Hold']

    Returns:
    - np.array of final overridden strategy predictions
    """
    final_preds = []

    for i, row in df.iterrows():
        if (row['Supertrend_Direction'] == 1) and (row['RSI'] > 45) and (row['ADX'] > 25):
            final_preds.append("Buy")
        elif row['Supertrend_Direction'] == -1 and (row['RSI'] > 45) and (row['ADX'] > 25):
            final_preds.append("Sell")
        else:
            final_preds.append("Hold")
    
    return np.array(final_preds)


In [79]:
data=pd.read_csv("C:\\Users\\vamsi\\OneDrive\\Desktop\\Documents\\studies\\complete stock market predictor\\Stock-Market-Predictor\\project_root\\data\\processed_data.csv")

In [89]:
adx = ADXIndicator(high=data['high'], low=data['low'], close=data['close'], window=14)
data['ADX'] = adx.adx()
data = calculate_SuperTrend(data)
data['RSI'] = RSIIndicator(close=data['close'], window=14).rsi()

HELLO


In [94]:
ok=apply_strategy_override(data)


In [95]:
print("Accuracy:", accuracy_score(ok, data['result']))
print("F1 Score:", f1_score(ok, data['result'], average='weighted'))
print("Confusion Matrix:\n", confusion_matrix(ok, data['result']))

Accuracy: 0.30309793768413534
F1 Score: 0.35735274508475007
Confusion Matrix:
 [[ 997  764  754]
 [2823 2253 2950]
 [ 253  262  145]]


## Saving the model

In [209]:
! pip install joblib

In [111]:
import joblib

# Suppose your model is named: rf_model
joblib.dump(meta_model, 'ensemble_model.pkl')


['ensemble_model.pkl']

In [168]:
import joblib

# Suppose your model is named: rf_model
joblib.dump(rf_model, 'random_forest_model_new.pkl')


['random_forest_model_new.pkl']

In [204]:
import joblib

# Suppose your model is named: rf_model
joblib.dump(meta_model, 'ensemble_model.joblib')

['ensemble_model.joblib']

In [200]:
joblib.dump(final_xgb, 'xgb_model.pkl')

['xgb_model.pkl']